# Chat group examples

### SDK option 1

In [ ]:
from promptflow import ChatGroup, ChatAgent

copiplot_agent = ChatAgent(name="copilot_flow", flow="./promotflow_copilot")
simulation_agent = ChatAgent(name="simulation_flow", flow="./promptflow_simulation")
# for eager flow, may use below
# simulation_agent = ChatAgent(name="simulation_flow", eager_flow=entry_func)

with ChatGroup(
    max_turns=10,
    max_token=5000,
    max_time=600,  # 10 minutes
    entry_agent=copiplot_agent,  # the first agent to speak
    speak_order=[copiplot_agent, simulation_agent],  # other than specifying, can also be "LLM" or "AUTO"(default)
) as chat_group:
    copiplot_agent.io_mapping(
        question=simulation_agent.outputs.generated_question, 
        model="gpt4",
    )
    simulation_agent.io_mapping(
        persona="Tom", 
        last_answer=copiplot_agent.outputs.output,
        chat_history=chat_group.chat_history,  # chat_history is a group-level context
        model="gpt4",
        goal=data_column,  # not sure the original purpose
    )
    # simulation_agent.terminate_func = lambda x: x == "<END>"

    # kick off the chat
    chat_group.run()

    # access to the agent outputs
    print("Chat history:", chat_group.chat_history)
    print("Last copilot flow output:", copiplot_agent.outputs["output"])
    print("Last simulation flow output:", simulation_agent.outputs["generated_question"])

### SDK option 2

In [ ]:
from promptflow import ChatGroup, ChatAgent

copiplot_agent = ChatAgent(name="copilot_flow", flow="./promotflow_copilot")
simulation_agent = ChatAgent(name="simulation_flow", flow="./promptflow_simulation")

chat_group = ChatGroup(
    agents=[copiplot_agent, simulation_agent],
    max_turns=10,
    max_token=5000,
    max_time=600,  # 10 minutes
    entry_agent=copiplot_agent,  # the first agent to speak
    speak_order=[copiplot_agent, simulation_agent],  # other than specifying, can also be "LLM" or "AUTO"(default)
    io_mapping = {
        "copilot_flow.question": "${simulation_flow.outputs.generated_question}",
        "copilot_flow.model": "gpt4",  # model is an external parameter
        "simulation_flow.persona": "Tom", 
        "simulation_flow.chat_history": "${group.chat_history}",  # chat_history is a group-level context
        "simulation_flow.model": "gpt4",
        "simulation_flow.goal": "<jsonl_data.column1>",  # not sure the original purpose
    }
)

# kick off the chat
chat_group.run()

# access to the agent outputs
print("Chat history:", chat_group.chat_history)
print("Last copilot flow output:", copiplot_agent.outputs["output"])
print("Last simulation flow output:", simulation_agent.outputs["generated_question"])


## Original yaml in experiment
```yaml
# multi turn conversation is described as a chat group, which contains a copilot flow and question simulation flow
  - name: multi_turn_chat
    type: chat_group
    max_turns: 5
    agents:
      - name: copilot_flow
        flow: ../copilot/promotflow_copilot/flow.dag.yaml
        inputs:
          question: ${roles.simulation_flow.outputs.generated_question}
          chat_history: []
          model: ${inputs.model_name}
      - name: simulation_flow
        flow: ../evaluation/similarity.yaml
        inputs:
          persona: ${inputs.persona}
          model: ${inputs.model_name}
          goal: ${data.leo_min_set.query}
          chat_history: ${group.chat_history}
```

### Yaml option 1
```yaml
# multi turn conversation is described as a chat group, which contains a copilot flow and question simulation flow
  - name: multi_turn_chat
    type: chat_group
    max_turns: 10
    max_token: 5000
    max_time: 600  # 10 minutes
    entry_agent: copilot_flow
    speak_order: [copilot_flow, simulation_flow]
    agents:
      - name: copilot_flow
        flow: ../copilot/promotflow_copilot/flow.dag.yaml
        inputs:
          question: ${roles.simulation_flow.outputs.generated_question}
          chat_history: []
          model: ${inputs.model_name}
      - name: simulation_flow
        flow: ../evaluation/similarity.yaml
        inputs:
          persona: ${inputs.persona}
          model: ${inputs.model_name}
          goal: ${data.leo_min_set.query}
          chat_history: ${group.chat_history}
```

### Yaml option 2

```yaml
# multi turn conversation is described as a chat group, which contains a copilot flow and question simulation flow
  - name: multi_turn_chat
    type: chat_group
    max_turns: 10
    max_token: 5000
    max_time: 600  # 10 minutes
    agents:
      - name: copilot_flow
        flow: ../copilot/promotflow_copilot/flow.dag.yaml
      - name: simulation_flow
        flow: ../evaluation/similarity.yaml
    entry_agent: copilot_flow
    speak_order: [copilot_flow, simulation_flow]
    io_mapping:
        copilot_flow.question: ${simulation_flow.outputs.generated_question}
        copilot_flow.model: ${inputs.model_name}
        copilot_flow.chat_history: []  # maybe omit?
        simulation_flow.persona: ${inputs.persona}
        simulation_flow.chat_history: ${group.chat_history}  # chat_history is a group-level context
        simulation_flow.model: ${inputs.model_name}
        simulation_flow.goal: ${data.leo_min_set.query}  # not sure the original purpose
      
```

### Terminate function

In [ ]:
from promptflow import ChatAgent

def terminate_chat(result):
    # result is the flow result that should be a dict
    return result["score"] >= 10

simulation_agent = ChatAgent(
    name="simulation_flow", 
    flow="./promptflow_simulation",
    terminate_func=terminate_chat,  # set customized terminate function
)

# this is the default terminate function if not specified
def default_terminate_function(result):
    # this would require the flow result has only one output, not a dict
    return result["output"] == "<END>"

In [ ]:
# Option 1
[
    ["copilot_agent", "chat message 1"],
    ["simulation_agent", "chat message 2"],
    ["copilot_agent", "chat message 3"],
]

# Option 2
[
    "<copilot_agent>: chat message 1",
    "<simulation_agent>: chat message 2",
    "<copilot_agent>: chat message 3",
]

# or maybe save both


In [1]:
try:
    print("123")
finally:
    print("456")

123
456


In [ ]:
from promptflow import ChatGroup, ChatAgent, ChatManager

manager = ChatManager(template="template.jinja2", connection="<connection_name>")
chat_group = ChatGroup(manager=manager, max_turns=10, max_token=5000, max_time=600, ...)